# PROYECTO INDIVIDUAL 1

### 1) Importacion de librerias

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
import copy 
from sklearn import preprocessing

import json

### 2) Ingesta de datos

In [2]:
df = pd.read_csv("movies_dataset - movies_dataset.csv", sep=',', encoding='latin1')
df.head(5)

C:\Users\johan\AppData\Local\Temp\ipykernel_12164\2648682348.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_dataset - movies_dataset.csv", sep=',', encoding='latin1')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


##### Vista general de la base de datos

In [3]:
df.describe

<bound method NDFrame.describe of        adult                              belongs_to_collection    budget  \
0      FALSE  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      FALSE                                                NaN  65000000   
2      FALSE  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      FALSE                                                NaN  16000000   
4      FALSE  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  FALSE                                                NaN         0   
45462  FALSE                                                NaN         0   
45463  FALSE                                                NaN         0   
45464  FALSE                                                NaN         0   
45465  FALSE                                                NaN         0   

                                         

In [4]:
# dimenciones de la base de datos
df.shape

(45466, 24)

In [5]:
# Cantidad de valores únicos por columna
df.nunique()

adult                        5
belongs_to_collection     1698
budget                    1226
genres                    4069
homepage                  7673
id                       45436
imdb_id                  45417
original_language           92
original_title           43373
overview                 44306
popularity               44176
poster_path              45024
production_companies     22708
production_countries      2393
release_date             17336
revenue                   6863
runtime                    353
spoken_languages          1931
status                       6
tagline                  20282
title                    42277
video                        2
vote_average                92
vote_count                1820
dtype: int64

### 3) Transformacion de los datos

Transformaciones: Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, y solo estas, transformaciones a los datos:

A) Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

B) Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

C) Los valores nulos del campo release date deben eliminarse.

D) De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

E) Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

F) Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.

#### A) Columnas anidadas

In [6]:
import ast
import re

# Definimos una expresión regular para extraer el nombre de la colección
pattern = r"'name': '([^']*)'"

# Extraemos el nombre de la colección utilizando la expresión regular
df['collection_name'] = df['belongs_to_collection'].str.extract(pattern, flags=re.IGNORECASE)

# Imprimimos el DataFrame con la columna normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection


In [7]:
df.collection_name.unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [8]:
df.collection_name.nunique()

1644

In [9]:
# Crear una nueva columna para los registros duplicados
df['collection_individual'] = df['collection_name'].str.split(', ')

# Duplicar los registros
df = df.explode('collection_individual').reset_index(drop=True)

# Eliminar la columna original "collection_name"
df.drop('collection_name', axis=1, inplace=True)

In [10]:
df.collection_individual.unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [11]:
df.collection_individual.nunique()

1667

In [12]:
# Convertimos la cadena JSON en una lista de diccionarios
df['genres'] = df['genres'].apply(ast.literal_eval)

# Función personalizada para extraer los nombres de los géneros
def extract_genre_names(genres_list):
    genre_names = [genre['name'] for genre in genres_list]
    return ', '.join(genre_names)

# Aplicamos la función personalizada a cada elemento de la columna "genres"
df['genre'] = df['genres'].apply(extract_genre_names)

# Imprimimos el DataFrame con la columna de géneros normalizados
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_individual,genre
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family"
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,"Adventure, Fantasy, Family"
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy"
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,"Comedy, Drama, Romance"
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy


In [13]:
df.genre.unique()

array(['Animation, Comedy, Family', 'Adventure, Fantasy, Family',
       'Romance, Comedy', ..., 'Crime, Comedy, Action, Family',
       'Action, Mystery, Thriller, Horror',
       'Family, Animation, Romance, Comedy'], dtype=object)

In [14]:
df.genre.nunique()

4069

In [15]:
# Crear una nueva columna para los registros duplicados
df['genre_individual'] = df['genre'].str.split(', ')

# Duplicar los registros
df = df.explode('genre_individual').reset_index(drop=True)

# Eliminar la columna original "genre"
df.drop('genre', axis=1, inplace=True)

In [16]:
df.genre_individual.unique()

array(['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History',
       'Science Fiction', 'Mystery', 'War', 'Foreign', '', 'Music',
       'Documentary', 'Western', 'TV Movie', 'Carousel Productions',
       'Vision View Entertainment', 'Telescene Film Group Productions',
       'Aniplex', 'GoHands', 'BROSTA TV',
       'Mardock Scramble Production Committee', 'Sentai Filmworks',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel'],
      dtype=object)

In [17]:
df.genre_individual.nunique()

33

In [18]:
# Función personalizada para normalizar la columna "production_companies"
def normalize_production_companies(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_companies"
df['company'] = df['production_companies'].apply(normalize_production_companies)

# Imprimimos el DataFrame con la columna de compañías de producción normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios
1,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios
2,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios
3,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,"TriStar Pictures, Teitler Film, Interscope Com..."
4,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Fantasy,"TriStar Pictures, Teitler Film, Interscope Com..."


In [19]:
df.company.unique()

array(['Pixar Animation Studios',
       'TriStar Pictures, Teitler Film, Interscope Communications',
       'Warner Bros., Lancaster Gate', ...,
       'Westdeutscher Rundfunk (WDR), Working Title Films, 20th Century Fox Television, CanWest Global Communications',
       'Sine Olivia', 'Yermoliev'], dtype=object)

In [20]:
df.company.nunique()

22580

In [21]:
# Crear una nueva columna para los registros duplicados
df['company_individual'] = df['company'].str.split(', ')

# Duplicar los registros
df = df.explode('company_individual').reset_index(drop=True)

# Eliminar la columna original "company"
df.drop('company', axis=1, inplace=True)

In [22]:
df.company_individual.unique()

array(['Pixar Animation Studios', 'TriStar Pictures', 'Teitler Film', ...,
       'Pirie Productions', 'Sine Olivia', 'Yermoliev'], dtype=object)

In [23]:
df.company_individual.nunique()

23350

In [24]:
# Función personalizada para normalizar la columna "production_countries"
def normalize_production_countries(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_countries"
df['country'] = df['production_countries'].apply(normalize_production_countries)

# Imprimimos el DataFrame con la columna de países de producción normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company_individual,country
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America
1,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America
2,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America
3,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America
4,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,Teitler Film,United States of America


In [25]:
df.country.unique()

array(['United States of America', 'Germany, United States of America',
       'United Kingdom, United States of America', ...,
       'Poland, Czech Republic, Slovakia', 'Cuba, Germany, Spain',
       'Egypt, Italy, United States of America'], dtype=object)

In [26]:
df.country.nunique()

2387

In [27]:
# Crear una nueva columna para los registros duplicados
df['country_individual'] = df['country'].str.split(', ')

# Duplicar los registros
df = df.explode('country_individual').reset_index(drop=True)

# Eliminar la columna original "country"
df.drop('country', axis=1, inplace=True)

In [28]:
df.country_individual.unique()

array(['United States of America', 'Germany', 'United Kingdom', 'France',
       'Italy', 'Spain', 'China', 'Australia', 'South Africa', 'Canada',
       '', 'Switzerland', 'Belgium', 'Japan', 'Iran', 'Netherlands',
       'Hong Kong', 'Tunisia', 'Ireland', 'Dominican Republic', 'Croatia',
       'Russia', 'Macedonia', 'Austria', 'Taiwan', 'New Zealand',
       'Mexico', 'Poland', 'Peru', 'Cuba', 'Liechtenstein', 'Denmark',
       'Portugal', 'Finland', 'Sweden', 'Argentina', 'Iceland',
       'South Korea', 'Serbia', 'Hungary', 'Czech Republic', 'India',
       'Brazil', 'Greece', 'Congo', 'Senegal', 'Burkina Faso', 'Romania',
       'Philippines', 'Vietnam', 'Trinidad and Tobago', 'Bulgaria',
       'Chile', 'Norway', 'Kazakhstan', 'Algeria', 'Luxembourg',
       'Georgia', 'Ukraine', 'Botswana', 'Aruba', 'Israel', 'Turkey',
       'Ecuador', 'Lebanon', 'Morocco', 'Bosnia and Herzegovina',
       'Bahamas', 'Malaysia', 'Bhutan', 'Jamaica', 'Pakistan', 'Nepal',
       'Thailand', 'Nam

In [29]:
df.country_individual.nunique()

159

In [30]:
# Función personalizada para normalizar la columna "spoken_languages"
def normalize_spoken_languages(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "spoken_languages"
df['spoken_language'] = df['spoken_languages'].apply(normalize_spoken_languages)

# Imprimimos el DataFrame con la columna de lenguajes normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company_individual,country_individual,spoken_language
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English
2,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English
3,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America,"English, FranÃ§ais"
4,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,Teitler Film,United States of America,"English, FranÃ§ais"


In [31]:
df.spoken_language.unique()

array(['English', 'English, FranÃ§ais', 'English, EspaÃ±ol', ...,
       'svenska, Deutsch', 'Ø§Ù\x84Ø¹Ø±Ø¨Ù\x8aØ©, Polski',
       'Fulfulde, English'], dtype=object)

In [32]:
df.spoken_language.nunique()

1842

In [33]:
# Crear una nueva columna para los registros duplicados
df['language_individual'] = df['spoken_language'].str.split(', ')

# Duplicar los registros
df = df.explode('language_individual').reset_index(drop=True)

# Eliminar la columna original "spoken_language"
df.drop('spoken_language', axis=1, inplace=True)

In [34]:
df.language_individual.unique()

array(['English', 'FranÃ§ais', 'EspaÃ±ol', 'Deutsch',
       'PÑ\x83Ñ\x81Ñ\x81ÐºÐ¸Ð¹', 'Latin', 'Nederlands',
       'å¹¿å·\x9eè¯\x9d / å»£å·\x9eè©±', 'æ\x99®é\x80\x9aè¯\x9d',
       'Magyar', '', 'shqip', 'Italiano',
       'í\x95\x9cêµ\xadì\x96´/ì¡°ì\x84\xa0ë§\x90', 'Ù\x81Ø§Ø±Ø³Û\x8c',
       'Dansk', 'æ\x97¥æ\x9c¬èª\x9e', 'Ø§Ù\x84Ø¹Ø±Ø¨Ù\x8aØ©', 'Hrvatski',
       'Bosanski', 'RomÃ¢nÄ\x83', 'Bahasa indonesia', 'Bahasa melayu',
       'svenska', '×¢Ö´×\x91Ö°×¨Ö´×\x99×ª', 'Ä\x8ceskÃ½', 'Polski',
       'Gaeilge', 'Norsk', 'SlovenÄ\x8dina', 'Tiáº¿ng Viá»\x87t',
       'PortuguÃªs', 'à¤¹à¤¿à¤¨à¥\x8dà¤¦à¥\x80', 'CatalÃ\xa0',
       'Ã\x8dslenska', 'Afrikaans', 'Srpski', 'à¦¬à¦¾à¦\x82à¦²à¦¾',
       'Wolof', 'Cymraeg', 'à¸\xa0à¸²à¸©à¸²à¹\x84à¸\x97à¸¢', 'LatvieÅ¡u',
       'Kiswahili', 'Ð±Ñ\x8aÐ»Ð³Ð°Ñ\x80Ñ\x81ÐºÐ¸ ÐµÐ·Ð¸Ðº',
       'ÎµÎ»Î»Î·Î½Î¹ÎºÎ¬', 'TÃ¼rkÃ§e', 'suomi', 'Esperanto',
       'Ð£ÐºÑ\x80Ð°Ñ\x97Ð½Ñ\x81Ñ\x8cÐºÐ¸Ð¹',
       'á\x83¥á\x83\x90á\x83\xa0á\x83\x97á\x83£á\x83\x9aá\x83

In [35]:
df.language_individual.nunique()

75

In [36]:
df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'], inplace = True)
df.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,collection_individual,genre_individual,company_individual,country_individual,language_individual
0,FALSE,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English
1,FALSE,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English
2,FALSE,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English
3,FALSE,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America,English
4,FALSE,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,Adventure,TriStar Pictures,United States of America,FranÃ§ais


In [37]:
df.shape

(419666, 24)

In [38]:
df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection_individual',
       'genre_individual', 'company_individual', 'country_individual',
       'language_individual'],
      dtype='object')

In [39]:
# Se renombran las columnas con valores desanidados

df.rename(columns={ 'collection_individual': 'collection', 
                    'genre_individual': 'genre',
                    'company_individual': 'company',
                    'country_individual': 'country',
                    'language_individual': 'language'
}, inplace=True)

# se imprimen los resultados para hacerles control

df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection', 'genre', 'company',
       'country', 'language'],
      dtype='object')

#### B) Valores nulos

In [40]:
# Los valores nulos de los campos revenue, budget son rellenados por el número 0

df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)
print('Nulos en revenue',df.revenue.isna().sum())
print('Nulos en budget',df.revenue.isna().sum())

Nulos en revenue 0
Nulos en budget 0


#### C) Modificacion de valores nulos

In [41]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de 
# estreno.
# Le damos el formato de fecha a todos los valores de la columna y los que no, le asignamos la etiqueta de nulo
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Se asegura que todos los registros tengan el mismo formato de fecha AAAA-MM-DD
df['release_date'] = pd.to_datetime(df['release_date']).dt.strftime('%Y-%m-%d')

# se identifican la cantidad de datos nulos de la columna 'release_date' 
df['release_date'].isnull().sum()

191

In [42]:
# eliminamos los registros con nulos 
df = df.dropna(subset = ['release_date'])

# controlamos para verificar que no existan nulos
df['release_date'].isnull().sum()

0

In [43]:
# creamos una nueva columna 'release_year' que va a contener unicamnete el dato de año de la columna 'release_date' 
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year']=df['release_date'].dt.year

# hacemos control sobre los cambios
df[['release_date','release_year']].tail(10)

,release_date,release_year
419654,1991-05-13,1991
419655,1991-05-13,1991
419656,1991-05-13,1991
419657,1991-05-13,1991
419660,2011-11-17,2011
419661,2003-08-01,2003
419662,2003-08-01,2003
419663,2003-08-01,2003
419664,1917-10-21,1917
419665,2017-06-09,2017


In [44]:
# eliminacion de datos nulos 
#df = df.dropna(subset=['release_date'])

# Transformamos el formato de fecha de 'release_date' a 'aaaa-mm-dd' si el formato es incorrecto
#df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Completamos los valores nulos o ceros en 'release_date' con 'sin dato'
#df['release_date'].fillna('sin dato', inplace=True)

# Creamos la columna 'release_year' extrayendo el año de 'release_date'
#df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
#df['release_year'].fillna(np.nan, inplace=True)
#df['release_year'] = df['release_year'].astype('Int64')

#Se impremen las columnas para realizar control de los cambios
#df[['release_date','release_year']]

#### D) Creacion de nuevas columnas

In [45]:
# Creamos la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
# cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Calculamos el retorno de inversión

# Convertimos 'revenue' y 'budget' al tipo numérico (float)
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Calculamos el retorno de inversión
df['return'] = df['revenue'] / df['budget']
df['return'].fillna(0, inplace=True)
df['return']

0         12.451801
1         12.451801
2         12.451801
3          4.043035
4          4.043035
            ...    
419661     0.000000
419662     0.000000
419663     0.000000
419664     0.000000
419665     0.000000
Name: return, Length: 419475, dtype: float64

In [46]:
df['return'].isna().sum()

0

In [47]:
df['return'].unique()

array([12.4518011,  4.0430346,  0.       , ...,  1.660765 ,  0.6343965,
        0.2826   ])

#### E) Eliminacion de columnas innecesarias

In [48]:
# Eliminamos las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage
df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], inplace = True)
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'collection', 'genre', 'company', 'country', 'language',
       'release_year', 'return'],
      dtype='object')

In [49]:
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,collection,genre,company,country,language,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995,12.451801
1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995,12.451801
2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English,1995,12.451801
3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,Adventure,TriStar Pictures,United States of America,English,1995,4.043035
4,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,Adventure,TriStar Pictures,United States of America,FranÃ§ais,1995,4.043035


In [50]:
df.shape

(419475, 19)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 419475 entries, 0 to 419665
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   budget             419475 non-null  int64         
 1   id                 419475 non-null  object        
 2   original_language  419379 non-null  object        
 3   overview           416976 non-null  object        
 4   popularity         419475 non-null  object        
 5   release_date       419475 non-null  datetime64[ns]
 6   revenue            419475 non-null  float64       
 7   runtime            418931 non-null  float64       
 8   status             419332 non-null  object        
 9   tagline            227683 non-null  object        
 10  title              419475 non-null  object        
 11  vote_average       419475 non-null  float64       
 12  collection         53658 non-null   object        
 13  genre              419475 non-null  object  

### 4. Desarrollo de API

Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

A) def peliculas_mes(mes): '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes (nombre del mes, en str, ejemplo 'enero') historicamente''' return {'mes':mes, 'cantidad':respuesta}

B) def peliculas_dia(dia): '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia (de la semana, en str, ejemplo 'lunes') historicamente''' return {'dia':dia, 'cantidad':respuesta}

C) def franquicia(franquicia): '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio''' return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}

D) def peliculas_pais(pais): '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo''' return {'pais':pais, 'cantidad':respuesta}

E) def productoras(productora): '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron''' return {'productora':productora, 'ganancia_total':respuesta, 'cantidad':respuesta}

F) def retorno(pelicula): '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo''' return {'pelicula':pelicula, 'inversion':respuesta, 'ganacia':respuesta,'retorno':respuesta, 'anio':respuesta}

G) def recomendacion('titulo'): '''Ingresas un nombre de pelicula y te recomienda las similares en una lista de 5 valores''' return {'lista recomendada': respuesta}

A) def peliculas_mes(mes):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''
    return {'mes':mes, 'cantidad':respuesta}

B) def peliculas_dia(dia):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente'''
    return {'dia':dia, 'cantidad':respuesta}

C) def franquicia(franquicia):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''
    return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}

D) def peliculas_pais(pais):
    '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''
    return {'pais':pais, 'cantidad':respuesta}

E) def productoras(productora):
    '''Ingresas la productora, retornando la ganancia toal y la cantidad de peliculas que produjeron'''
    return {'productora':productora, 'ganancia_total':respuesta, 'cantidad':respuesta}

F) def retorno(peliculaa):
    '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo'''
    return {'pelicula':pelicula, 'inversion':respuesta, 'ganacia':respuesta,'retorno':respuesta, 'anio':respuesta}

M.L.

G) ef recomendacion('titulo'):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    return {'lista recomendada': respuesta}

#### A)

In [52]:
# Funcion de consulta de la cantidad de peliculas por mes
def peliculas_mes(mes):

    mes = mes.lower()
    meses = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12}

    mes_numero = meses[mes]

    # Convertir la columna "fecha" a un objeto de tipo fecha
    df['release_date'] = pd.to_datetime(df['release_date'])


    try:
        month_filtered = df[df['release_date'].dt.month == mes_numero]
    except (ValueError, KeyError, TypeError):
        return None

    month_unique = month_filtered.drop_duplicates(subset='id')
    respuesta = month_unique.shape[0]

    return {'mes':mes, 'cantidad':respuesta}

In [76]:
#Control sobre la funcion
peliculas_mes('abril')

{'mes': 'abril', 'cantidad': 3452}

#### B)

In [54]:
# Funcion de consulta de peliculas por dia de la semana 

def peliculas_dia(dia):

    days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miercoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sabado': 'Saturday',
    'domingo': 'Sunday'}

    day = days[dia.lower()]

    lista_peliculas_day = df[df['release_date'].dt.day_name() == day].drop_duplicates(subset='id')
    cantidad = lista_peliculas_day.shape[0]

    return {'dia': dia, 'cantidad': cantidad}

In [79]:
# Control sobre la funcion 
peliculas_dia("sabado")

{'dia': 'sabado', 'cantidad': 5149}

#### C)

In [81]:
df['collection'].unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [57]:
# Funcion de consulta del numero de colecciones de peliculas, su ganancia total y ganancia promedio 
def franquicia(franquicia):
    # productora = company?
    # franquicia = collection?

    lista_pelis_franquicia = df[(df['collection'] == franquicia)].drop_duplicates(subset='id')

    cantidad_pelis_franq = (lista_pelis_franquicia).shape[0]
    revenue_franq = lista_pelis_franquicia['revenue'].sum()
    promedio_franq = revenue_franq/cantidad_pelis_franq

    return {'franquicia':franquicia, 'cantidad':cantidad_pelis_franq, 'ganancia_total':revenue_franq, 'ganancia_promedio':promedio_franq}

In [85]:
# Control sobre la funcion 
franquicia("Toy Story Collection")

{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1937890605.0,
 'ganancia_promedio': 645963535.0}

#### D)

In [135]:
# Funcion de consulta del numero de peliculas por pais 1
'''
def peliculas_pais(pais):
    peliculas_pais = df[df['country'].apply(lambda x: pais in x)].drop_duplicates(subset='id')
    respuesta = len(peliculas_pais)
    return {'pais': pais, 'cantidad de peliculas': respuesta}
'''


In [122]:
# Funcion de consulta del numero de peliculas por pais 2
def peliculas_pais(pais):
    # Selecciona todas las películas del DataFrame 'df' cuya columna 'country' contiene el país especificado.
    # La función lambda se utiliza para aplicar la operación a cada valor de la columna 'country'.
    peliculas_pais = df[df['country'].apply(lambda x: pais in str(x) if pd.notnull(x) else False)]
    
    # Elimina las filas duplicadas de la película para evitar contar varias veces una misma película.
    peliculas_pais = peliculas_pais.drop_duplicates(subset='id')
    
    # Cuenta la cantidad de películas restantes después de eliminar los duplicados.
    respuesta = len(peliculas_pais)
    
    # Devuelve los resultados en un diccionario con claves legibles.
    return {'pais': pais, 'cantidad de peliculas': respuesta}

In [136]:
# Control sobre la funcion
peliculas_pais('Italy')

{'pais': 'Italy', 'cantidad de peliculas': 2165}

#### E)

In [61]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'collection', 'genre', 'company', 'country', 'language',
       'release_year', 'return'],
      dtype='object')

In [89]:
df['company'].unique()

array(['Pixar Animation Studios', 'TriStar Pictures', 'Teitler Film', ...,
       'Pirie Productions', 'Sine Olivia', 'Yermoliev'], dtype=object)

In [62]:
# Funcion de consulta del numero de peliculas por productora, ganancias totales y numero de peliculas

def productoras(productora):

    lista_pelis_productoras = df[(df['company'] == productora)].drop_duplicates(subset='id')

    cantidad_pelis_prod = (lista_pelis_productoras).shape[0]
    revenue_prod = lista_pelis_productoras['revenue'].sum()


    return {'productora':productora, 'ganancia_total':revenue_prod, 'cantidad':cantidad_pelis_prod}

In [92]:
# control sobre la funcion
productoras('Sine Olivia')

{'productora': 'Sine Olivia', 'ganancia_total': 0.0, 'cantidad': 1}

#### F)

In [94]:
df['title'].unique()

array(['Toy Story', 'Jumanji', 'Grumpier Old Men', ...,
       'Century of Birthing', 'Satan Triumphant', 'Queerama'],
      dtype=object)

In [133]:
# Funcion de consulta de peliculas, su inversion, ganancias, retorno y año
'''
def retorno(pelicula):
    pelicula_info = df[df['title'] == pelicula].iloc[0]
    #pelicula_info = df[(df['title'] == pelicula)].drop_duplicates(subset='id')
    inversion = pelicula_info['budget']
    ganancia = pelicula_info['revenue']
    retorno = pelicula_info['return']
    anio = pelicula_info['release_year']
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'año': anio}
'''

In [137]:
def retorno(pelicula):
    info_pelicula = df[(df['title'] == pelicula)].drop_duplicates(subset='title')
    pelicula_nombre = info_pelicula['title'].iloc[0]
    inversion_pelicula = str(info_pelicula['budget'].iloc[0])
    ganancia_pelicula = str(info_pelicula['revenue'].iloc[0])
    retorno_pelicula = str(info_pelicula['return'].iloc[0])
    year_pelicula = str(info_pelicula['release_year'].iloc[0])

    return {'pelicula':pelicula_nombre, 'inversion':inversion_pelicula, 'ganacia':ganancia_pelicula,'retorno':retorno_pelicula, 'anio':year_pelicula}

In [140]:
# Control sobre la funcion
retorno('Toy Story')

{'pelicula': 'Toy Story',
 'inversion': '30000000',
 'ganacia': '373554033.0',
 'retorno': '12.4518011',
 'anio': '1995'}

In [139]:
df['title'].unique()

array(['Toy Story', 'Jumanji', 'Grumpier Old Men', ...,
       'Century of Birthing', 'Satan Triumphant', 'Queerama'],
      dtype=object)

In [67]:
df.shape

(419475, 19)

In [75]:
#df.to_csv('movies_ok_1.csv', index=False)